In [15]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

import supervision as sv
import cv2

from mediapipe_model_maker import object_detector

import random
import shutil

In [10]:
# Define the path to the folder containing the individual JSON files
folder_path = "android_figurine/Blue_ball"
output_file = "android_figurine/Blue_ball/labels.json"

# Define the category mapping for blue_ball
category_mapping = {
    "blue_ball": 1  # blue_ball with category_id 1
}

# Initialize the combined data structure
combined_data = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 0, "name": "background"},
        {"id": 1, "name": "blue_ball"}
    ]
}

# Initialize counters for image and annotation IDs
image_id_counter = 0
annotation_id_counter = 0

# Iterate through each JSON file in the folder
for json_file in os.listdir(folder_path):
    if json_file.endswith(".json"):
        with open(os.path.join(folder_path, json_file), "r") as file:
            data = json.load(file)
            
            # Check if data is a list
            if isinstance(data, list):
                # Process the list of dictionaries
                for entry in data:
                    # Get image file name
                    image_file_name = entry.get("image", None)
                    
                    if not image_file_name:
                        print(f"Skipping file {json_file}: 'image' key not found.")
                        continue
                    
                    # Add image information
                    combined_data["images"].append({
                        "id": image_id_counter,
                        "file_name": image_file_name
                    })
                    
                    # Process annotations
                    annotations = entry.get("annotations", [])
                    for annotation in annotations:
                        label = annotation.get("label", None)
                        coordinates = annotation.get("coordinates", None)
                        
                        if not label or not coordinates:
                            print(f"Skipping annotation in {json_file}: 'label' or 'coordinates' key not found.")
                            continue
                        
                        # Map the label (always "blue_ball") to category_id 1
                        category_id = category_mapping.get(label, 0)  # Should map to 1 (blue_ball)
                        
                        # Extract bounding box in COCO format [x_min, y_min, width, height]
                        bbox = [
                            coordinates["x"] - coordinates["width"] / 2,  # x_min
                            coordinates["y"] - coordinates["height"] / 2,  # y_min
                            coordinates["width"],  # width
                            coordinates["height"]  # height
                        ]
                        
                        # Append the annotation to combined data
                        combined_data["annotations"].append({
                            "image_id": image_id_counter,
                            "bbox": bbox,
                            "category_id": category_id
                        })
                        
                        # Increment annotation ID
                        annotation_id_counter += 1
                    
                    # Increment image ID
                    image_id_counter += 1
            else:
                print(f"Skipping file {json_file}: Expected a list but got {type(data)}")

# Save the combined JSON file
with open(output_file, "w") as output:
    json.dump(combined_data, output, indent=4)

print(f"Combined JSON saved to {output_file}")

Skipping file labels.json: Expected a list but got <class 'dict'>
Combined JSON saved to android_figurine/Blue_ball/labels.json


In [14]:
#Augmentation here


In [13]:
with open(os.path.join('android_figurine/Blue_ball', "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")


0: background
1: blue_ball


In [18]:
# Paths
original_images_path = "android_figurine/Blue_ball"
train_images_path = "android_figurine/Blue_ball/train"
val_images_path = "android_figurine/Blue_ball/val"
labels_path = "android_figurine/Blue_ball/labels.json"

# Load labels.json
with open(labels_path, 'r') as file:
    data = json.load(file)

# Create Training and Validation Folders if they don't exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)

# List of image filenames
image_filenames = [img["file_name"] for img in data["images"]]

# Shuffle and split
random.shuffle(image_filenames)
split_idx = int(len(image_filenames) * 0.8)  # 80% train, 20% val

train_filenames = image_filenames[:split_idx]
val_filenames = image_filenames[split_idx:]

# Move Images
for filename in train_filenames:
    src = os.path.join(original_images_path, filename)
    dst = os.path.join(train_images_path, filename)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"File not found: {src}")

for filename in val_filenames:
    src = os.path.join(original_images_path, filename)
    dst = os.path.join(val_images_path, filename)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"File not found: {src}")

# Update labels.json
train_data = {"images": [], "annotations": [], "categories": data["categories"]}
val_data = {"images": [], "annotations": [], "categories": data["categories"]}

train_image_ids = set()
val_image_ids = set()

for img in data["images"]:
    if img["file_name"] in train_filenames:
        train_data["images"].append(img)
        train_image_ids.add(img["id"])
    elif img["file_name"] in val_filenames:
        val_data["images"].append(img)
        val_image_ids.add(img["id"])

for ann in data["annotations"]:
    if ann["image_id"] in train_image_ids:
        train_data["annotations"].append(ann)
    elif ann["image_id"] in val_image_ids:
        val_data["annotations"].append(ann)

# Save updated JSON files
with open("android_figurine/Blue_ball/train_labels.json", 'w') as file:
    json.dump(train_data, file, indent=4)

with open("android_figurine/Blue_ball/val_labels.json", 'w') as file:
    json.dump(val_data, file, indent=4)



In [25]:
train_data = object_detector.Dataset.from_coco_folder("android_figurine/Blue_ball/train/", cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(val_images_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)


INFO:tensorflow:On image 0


2024-09-10 17:12:03.993093: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 17:12:04.244785: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:On image 0
train_data size:  44
validation_data size:  12


In [26]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [29]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

INFO:tensorflow:Training the models...


Epoch 1/30


5/5 [==============================] - 38s 2s/step - total_loss: 10.2751 - cls_loss: 10.1442 - box_loss: 0.0014 - model_loss: 10.2122 - val_total_loss: 1.3151 - val_cls_loss: 1.2155 - val_box_loss: 7.3354e-04 - val_model_loss: 1.2522
Epoch 2/30
5/5 [==============================] - 10s 2s/step - total_loss: 1.3414 - cls_loss: 1.1991 - box_loss: 0.0016 - model_loss: 1.2785 - val_total_loss: 1.2018 - val_cls_loss: 1.1035 - val_box_loss: 7.0807e-04 - val_model_loss: 1.1389
Epoch 3/30
5/5 [==============================] - 11s 2s/step - total_loss: 1.1976 - cls_loss: 1.0826 - box_loss: 0.0010 - model_loss: 1.1347 - val_total_loss: 1.0677 - val_cls_loss: 0.9720 - val_box_loss: 6.5796e-04 - val_model_loss: 1.0049
Epoch 4/30
5/5 [==============================] - 11s 2s/step - total_loss: 1.0089 - cls_loss: 0.8918 - box_loss: 0.0011 - model_loss: 0.9461 - val_total_loss: 0.8503 - val_cls_loss: 0.7446 - val_box_loss: 8.5616e-04 - val_model_loss: 0.7874
Epoch 5/30
5/5 [========================

In [30]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


3/3 [==============================] - 2s 124ms/step - total_loss: 0.3262 - cls_loss: 0.2352 - box_loss: 5.6253e-04 - model_loss: 0.2633
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.674
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.675
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.800
 Average Recall     (AR)

In [31]:
model.export_model()

Exporting a floating point model


INFO:tensorflow:Assets written to: /tmp/tmpstasrbui/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpstasrbui/saved_model/assets
2024-09-10 18:16:56.876501: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-10 18:16:56.876532: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-10 18:16:56.877188: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpstasrbui/saved_model
2024-09-10 18:16:56.954826: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-10 18:16:56.954852: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpstasrbui/saved_model
2024-09-10 18:16:57.105449: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-09-10 18:16:57.161195: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-09-10 18:16:58.068677: I tensorflow/cc/saved_model/

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model.tflite


In [10]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")
# Quantization seems to significiantly reduce results! Skip quantization for the real model

/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Epoch 1/10


/home/jesse/.local/lib/python3.8/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


15/15 [==============================] - 65s 1s/step - total_loss: 2.7510 - cls_loss: 1.1556 - box_loss: 0.0319 - model_loss: 2.7510 - val_total_loss: 2.6202 - val_cls_loss: 1.4536 - val_box_loss: 0.0233 - val_model_loss: 2.6202
Epoch 2/10
15/15 [==============================] - 12s 831ms/step - total_loss: 1.8082 - cls_loss: 0.9480 - box_loss: 0.0172 - model_loss: 1.8082 - val_total_loss: 2.4070 - val_cls_loss: 1.4715 - val_box_loss: 0.0187 - val_model_loss: 2.4070
Epoch 3/10
15/15 [==============================] - 13s 851ms/step - total_loss: 1.3004 - cls_loss: 0.7828 - box_loss: 0.0104 - model_loss: 1.3004 - val_total_loss: 2.3420 - val_cls_loss: 1.4476 - val_box_loss: 0.0179 - val_model_loss: 2.3420
Epoch 4/10
15/15 [==============================] - 12s 829ms/step - total_loss: 1.1620 - cls_loss: 0.7007 - box_loss: 0.0092 - model_loss: 1.1620 - val_total_loss: 1.9933 - val_cls_loss: 1.2994 - val_box_loss: 0.0139 - val_model_loss: 1.9933
Epoch 5/10
15/15 [========================

In [11]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_2 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

15/15 [==============================] - 64s 1s/step - total_loss: 2.6565 - cls_loss: 1.1689 - box_loss: 0.0298 - model_loss: 2.6565 - val_total_loss: 1.8481 - val_cls_loss: 1.1372 - val_box_loss: 0.0142 - val_model_loss: 1.8481
Epoch 2/15
15/15 [==============================] - 12s 821ms/step - total_loss: 2.0396 - cls_loss: 1.0502 - box_loss: 0.0198 - model_loss: 2.0396 - val_total_loss: 1.5235 - val_cls_loss: 0.9923 - val_box_loss: 0.0106 - val_model_loss: 1.5235
Epoch 3/15
15/15 [==============================] - 12s 791ms/step - total_loss: 1.7346 - cls_loss: 0.9916 - box_loss: 0.0149 - model_loss: 1.7346 - val_total_loss: 1.4391 - val_cls_loss: 0.9336 - val_box_loss: 0.0101 - val_model_loss: 1.4391
Epoch 4/15
15/15 [==============================] - 12s 822ms/step - total_loss: 1.5603 - cls_loss: 0.9261 - box_loss: 0.0127 - model_loss: 1.5603 - val_total_loss: 1.4560 - val_cls_loss: 0.9483 - val_box_loss: 0.0102 - val_model_loss: 1.4560
Epoch 5/15
15/15 [========================

In [12]:
model.export_model('model_int8_qat.tflite')

Exporting a qat int8 model


/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmp47g6p_8q/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp47g6p_8q/saved_model/assets
2024-09-06 18:06:38.124848: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-06 18:06:38.124872: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-06 18:06:38.125055: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp47g6p_8q/saved_model
2024-09-06 18:06:38.228674: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-06 18:06:38.228708: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp47g6p_8q/saved_model
2024-09-06 18:06:38.538127: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-09-06 18:06:40.583126: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp47g6p_8q/saved_model
2024-09-06 18:06:41.327456: I ten

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_int8_qat.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_int8_qat.tflite


In [32]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_float16()
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)

Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_2 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

INFO:tensorflow:Assets written to: /tmp/tmp6g29qctw/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp6g29qctw/saved_model/assets
2024-09-10 18:19:06.316589: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-10 18:19:06.316616: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-10 18:19:06.316806: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp6g29qctw/saved_model
2024-09-10 18:19:06.395740: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-10 18:19:06.395767: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp6g29qctw/saved_model
2024-09-10 18:19:06.599052: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-09-10 18:19:07.510692: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp6g29qctw/saved_model
2024-09-10 18:19:07.916687: I ten

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_fp16.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_fp16.tflite


In [33]:
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
# The post-training quantization step seems to have no effect on the model evaluation, but it should make the model faster so ALWAYS include this step!

12/12 [==============================] - 4s 75ms/step - total_loss: 0.3854 - cls_loss: 0.2944 - box_loss: 5.6240e-04 - model_loss: 0.3226
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.674
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.675
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.800
 Average Recall     (AR